# KimCNN 2

In [1]:
import logging
from nltk import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchtext import data, datasets

from utils import *

In [2]:
MAX_VOCAB_SIZE = 25000
BATCH_SIZE = 8
EPOCHS = 5

EMBEDDING_TYPE = "glove.6B.100d"
#TODO: ergänzen
embeddings_dict = {"glove.6B.100d": "glove"}
EMBEDDING_NAME = "unknown"
if EMBEDDING_TYPE in embeddings_dict.keys():
    EMBEDDING_NAME = embeddings_dict[EMBEDDING_TYPE]


corpus = pd.read_csv("../corpora/small_amazon_reviews_electronic.csv")

In [3]:
REVIEW = data.Field(tokenize = "toktok",
                    lower = True)

RATING = data.LabelField()
assigned_fields = {"review": ('text', REVIEW), 
                   "rating": ('label', RATING)}

train_data, val_data, test_data = data.TabularDataset.splits(path="../corpora/splits/", 
                                                              train='train.json',
                                                              validation='val.json', 
                                                              test='test.json', 
                                                              format='json',
                                                              fields=assigned_fields,
                                                              skip_header = True)


REVIEW.build_vocab(train_data, 
                   vectors = EMBEDDING_TYPE, 
                   unk_init = torch.Tensor.normal_,
                   max_size = MAX_VOCAB_SIZE)
RATING.build_vocab(train_data)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    logging.info("GPU will be used.")


train_iterator, val_iterator, test_iterator = data.BucketIterator.splits((train_data, val_data, test_data), 
                                                                         batch_size = BATCH_SIZE,
                                                                         device = device,
                                                                         sort_key = lambda x: len(x.text),
                                                                         sort = False,
                                                                         sort_within_batch=False)
